In [39]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# The idea

First, retrieve all scene objects' configuration and position. 

In [40]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [41]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [42]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [43]:
for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

[-1.72, 0.25, 0.25]
[-0.35, 1.5, 0.5]
[-1.52, -1.22, 0.5]
[-0.15, -0.1, 0.25]
[1.4, -0.15, 0.5]
[-2.02, 1.73, 0.15]


In [44]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [45]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
- Get the bounding box vertices from $x_{min}, x_{max}, y_{min}, y_{max}, z_{min}, z_{max}$
- Map bounding box vertices (in object's reference frame to global reference frame)
- Test applicability using the normal's orientation (i think i'll need to check sine and cossine, or limit the orientation within [0,2pi] )
- Create Configuration Vertice 

In [46]:
handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,0)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [47]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14

-0.007493937796628352

In [48]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [33]:
for element in info_list:
    normals = get_normals_orientation(element['object_orientation'][2])
    # position_x = round
    print(f'{element["object_name"]}: {normals}')

Cuboid_0: (5.470312481296908e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_1: (-4.748175431334412e-19, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_2: (3.2353105017202745e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_3: (2.567217400476508e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_4: (-1.7059819275407946e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)


In [50]:
x_min, y_min, z_min, x_max, y_max, z_max = get_bounding_box_corners_positions(client_id,listazinha[0]['object_handler'],"model")

In [ ]:
def get_bounding_box_corners_local_coordinates(client_id, object_handle, parameter_id_type='model', consider_up_layer = False):

    """This method returns the coordinates of the bounding box corners, in the local reference frame. The order starts at the top right point and goes anti-clockwise until the down right point.

    Returns:
        tuple: vectors from point 1 to point 8
    """
    # the idea is to get the coordinates and return it anti-clockwise (to match mapping algorithm)
    x_min, y_min, z_min, x_max, y_max, z_max = get_bounding_box_corners_positions(client_id,object_handle,parameter_id_type)

    # down layer points
    point_1 = np.array([x_max,y_max,z_min])
    point_2 = np.array([x_min,y_max,z_min])
    point_3 = np.array([x_min,y_min,z_min])
    point_4 = np.array([x_max,y_min,z_min])
    
    if consider_up_layer:
        point_5 = np.array([x_max,y_max,z_max])
        point_6 = np.array([x_min,y_max,z_max])
        point_7 = np.array([x_min,y_min,z_max])
        point_8 = np.array([x_max,y_min,z_max])
        return point_1, point_2, point_3, point_4, point_5, point_6, point_7, point_8
    
    else:
        return point_1, point_2, point_3, point_4
